# Gaining familiarity with web3py

In this notebook I will work through the examples provided in the [web3py documentation](https://web3py.readthedocs.io/en/stable/examples.html) to get a better feeling for the functionality. The goal is to be able to compile, deploy and interact with Solidity contracts directly from within Python. This is important since we need this functionality to be able to dynamically dispatch and interact with contracts from within the server backend of the LUCE technical prototype.  

**Update:** Since the old way to deploy contracts via `w3.eth.contract.deploy()` is deprecated this notebook also contains experiments with the new contract creation methods as outlined in the contract section of the web3.py documentation [here](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=abi).

### Software used

Over the last weeks I developed LuceVM, an Ubuntu 16.04 virtual machine that contains all required software and servers. Using the VM allows for a reproducible development environment and abstracts away the layer of managing dependencies for the different packages. It also makes it easier to ensure all components interact correctly with each other.  

The most important tools for us at this stage are:
* Python 3.7
    * web3
    * solc-py-x
* Jupyter Notebook Server for interaction and documentation
* Virtual Environments via conda to handle dependencies
* Solidity Compiler
* Node.js
* Ganache

### Preparation: Connect Python to local Ganache Node

First we start an instance of Ganache, either via GUI or the  ```run_ganache.sh``` script in LuceVM.  
Update: Running Ganache is handled via `run_servers_tmux.sh` directly as part of the VM setup process now.

In [30]:
from web3 import Web3, HTTPProvider

# Ganache Connection
w3 = Web3(Web3.IPCProvider("/home/vagrant/.ethereum/rinkeby/geth.ipc"))
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

Alternatively we can also connect to Ganache running on the host machine:  
(Change settings to allow incoming connections from 192.168.72.1 or 0.0.0.0)

In [14]:
w3.isConnected()

True

### Obtain Accounts

In [2]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

InvalidURI: ParseResult(scheme='', netloc='', path='127.0.0.1:30303', params='', query='', fragment='') isn't a valid URI

In [7]:
# Display all 10 default accounts
accounts

['0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA',
 '0xbf7ca7e607516Cd0F1b94a6C77F45fbFD936949C',
 '0x43e196C418b4b7Ebf71ba534042cC8907bd39dc9']

In [8]:
# Display address of first account
accounts[0]

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

Store credentials of first account for testing:

In [9]:
# Wallet address
wallet_address       = "0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA"

# Private key (from Ganache interface)
wallet_private_key   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"

### Send ether

In [11]:
# Define a function to conveniently send ether
import time
def send_ether(amount_in_ether, recipient_address):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    # How many transactions have been made by wallet?
    nonce = w3.eth.getTransactionCount(wallet_address)
    
    # Specify transcation details
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, wallet_private_key)

    # Send transaction & store transaction hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    # Check if transaction was added to blockchain
    time.sleep(0.5)
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    return txn_hash

In [12]:
# Set recipient
recipient = accounts[1]

In [13]:
# Send ether and store transaction hash
txn_hash = send_ether(1.5,recipient)

In [14]:
# Count transactions performed by a given address
w3.eth.getTransactionCount(wallet_address)

5

### Check Balance

There are two accounts we are interested in:  
The sender: `wallet_address`  
And the recipient: `recpient`

In [15]:
# Check balance
w3.eth.getBalance(recipient)

101500000000000000000

In [16]:
# Balance in ether
w3.fromWei(w3.eth.getBalance(recipient), 'ether')

Decimal('101.5')

In [17]:
# Balance of our primary account
w3.fromWei(w3.eth.getBalance(wallet_address), 'ether')

Decimal('98.46921988')

In [18]:
tx_hash = send_ether(1, "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E")

In [19]:
w3.fromWei(w3.eth.getBalance("0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"), 'ether')

Decimal('1')

### Look up a block

In [20]:
w3.eth.getBlock(1)

AttributeDict({'number': 1,
 'hash': HexBytes('0x07904a959c3bd4d3ed9b9dee31390fbe79c97be2d274b8b3c6d0bd48ca416ad7'),
 'parentHash': HexBytes('0x967fb7c714324204fcb6f5820a1191838bdc08d5c4dce0e17456dab39d0520d7'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [21]:
# Retrieve the last block
w3.eth.getBlock('latest')

AttributeDict({'number': 6,
 'hash': HexBytes('0x78109ffd25389d3f92bef7d8342699279aa3087f81fb70d2f8930aa55c02f308'),
 'parentHash': HexBytes('0x956e7f703a9c3799cac6df5db68dc4e57833d7085b76495016030409f06f7342'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

In [22]:
# Directly obtain the current block number
w3.eth.blockNumber

6

### Loop up a transaction via hash

In [23]:
w3.eth.getTransaction(txn_hash)

AttributeDict({'hash': HexBytes('0x045bb049f70bfbe1986a9c16c7f60085ce54e2bbd23544c0a3aa49252f6149c6'),
 'nonce': 4,
 'blockHash': HexBytes('0x956e7f703a9c3799cac6df5db68dc4e57833d7085b76495016030409f06f7342'),
 'blockNumber': 5,
 'transactionIndex': 0,
 'from': '0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA',
 'to': '0xbf7ca7e607516Cd0F1b94a6C77F45fbFD936949C',
 'value': 1500000000000000000,
 'gas': 2000000,
 'gasPrice': 40000000000,
 'input': '0x',
 'v': 41,
 'r': HexBytes('0xe8f05a56b0e47ba19c6ebadf1639f24b6a3ac734d2bb72dda60922382f4e13a9'),
 's': HexBytes('0x516a674972b7e2e07de836a8116fea8b41fd0666481b46cd6e70bc386e695dfe')})

In [24]:
# Transaction receipt
w3.eth.getTransactionReceipt(txn_hash)

AttributeDict({'transactionHash': HexBytes('0x045bb049f70bfbe1986a9c16c7f60085ce54e2bbd23544c0a3aa49252f6149c6'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x956e7f703a9c3799cac6df5db68dc4e57833d7085b76495016030409f06f7342'),
 'blockNumber': 5,
 'from': '0x92d44e8579620f2db88a12e70fe38e8cdb3541ba',
 'to': '0xbf7ca7e607516cd0f1b94a6c77f45fbfd936949c',
 'gasUsed': 21000,
 'cumulativeGasUsed': 21000,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

### Compile contract

Important that py-solc is installed for code to execute.  
Update: No need to worry anymore -> LuceVM has all correct dependencies installed

In [25]:
#!{sys.executable} -m pip install py-solc

In [109]:
#!{sys.executable} -m pip install py-solc-x


NameError: name 'solcx' is not defined

In [18]:
import sys
import time
import pprint

from web3.providers.eth_tester import EthereumTesterProvider
from web3 import Web3
import solcx
from solcx import compile_source

def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()

    return compile_source(source, solc_version="0.6.2")

In [19]:
contract_source_path = './data/LuceRegistry.sol'

In [20]:
compiled_sol = compile_source_file('./data/Main.sol')

In [21]:
contract_id, contract_interface = compiled_sol.popitem()

In [22]:
contract_id

'<stdin>:LuceMain'

In [23]:
# Interface contains ABI and more
contract_interface["abi"]

[{'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'approved',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'uint256',
    'name': 'tokenId',
    'type': 'uint256'}],
  'name': 'Approval',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'operator',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'bool',
    'name': 'approved',
    'type': 'bool'}],
  'name': 'ApprovalForAll',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': False,
    'internalType': 'uint256',
    'name': 'tokenID',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'licenseType',
    'type': 'uint256'},
   {'indexed': False

Up to this point everything is working. We can compile the contract and also have the abi and binary code conveniently stored in our ```contract_interface```.

### Deploy Contract

Method 1:

In [24]:
DEBUG = True
# Define default account (used as default for 'from' property in transaction dictionary)
private_key = "52417fb192c8cb46bf2b76e814992a803910d42cd19ca0ae0a83c5de97c6dbd6"
public_key = w3.eth.account.privateKeyToAccount(private_key).address
CHAIN_ID = 4


In [25]:
private_key

'52417fb192c8cb46bf2b76e814992a803910d42cd19ca0ae0a83c5de97c6dbd6'

In [34]:
def sign_and_send(contract_txn, private_key, name):
    try:
        signed_txn = w3.eth.account.signTransaction(contract_txn, private_key)
      
        tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
      
    
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        transaction = receipt_to_dict(tx_receipt, name)
        
    except ValueError as e:
        if DEBUG:
            print()
            print(e)
        return [e,name]
    return transaction

def receipt_to_dict(tx_receipt, name):
    receipt = {}
    receipt["blockHash"] = tx_receipt.blockHash.hex()
    receipt["blockNumber"] = tx_receipt.blockNumber
    receipt["contractAddress"] = tx_receipt.contractAddress
    receipt["cumulativeGasUsed"] = tx_receipt.cumulativeGasUsed
    receipt["effectiveGasPrice"] = w3.toInt(hexstr = tx_receipt.effectiveGasPrice)
    receipt["from"] = tx_receipt["from"]
    receipt["gasUsed"] = tx_receipt.gasUsed
    receipt["logsBloom"] = tx_receipt.logsBloom.hex()
    receipt["status"] = tx_receipt.status
    receipt["to"] = tx_receipt.to
    receipt["transactionHash"] = tx_receipt.transactionHash.hex()
    receipt["transactionIndex"] = tx_receipt.transactionIndex
    receipt["type"] = tx_receipt.type
    receipt["fees"] =  receipt["effectiveGasPrice"] * receipt["gasUsed"]
    receipt["transaction name"] = name

    return receipt

In [35]:
# Store abi and bytecode of contract
abi = contract_interface['abi']
bytecode = contract_interface['bin']

In [38]:
# Create contract object
my_contract = w3.eth.contract(abi=abi,bytecode=bytecode)
nonce = w3.eth.getTransactionCount(public_key)
print(nonce)
txn_dict = {
        'from': public_key,
        'chainId': CHAIN_ID,
        'gasPrice': w3.toWei('20', 'gwei'),
        'nonce': nonce,
        }
#gas = w3.eth.estimateGas(contract_txn)ù
gas = my_contract.constructor().estimateGas()

txn_dict["gas"]=gas*2

contract_txn = my_contract.constructor().buildTransaction(txn_dict)

tx_receipt = sign_and_send(contract_txn, private_key, "deployment of Test")


183


In [39]:
tx_receipt

{'blockHash': '0x13f032638db10af1d2fc42b0daad86bb51da4dd70cccbd69947162425a6bd5b7',
 'blockNumber': 10075803,
 'contractAddress': '0x6393398AeBD4B82f262C7A89ED68C44494eef2f0',
 'cumulativeGasUsed': 5788951,
 'effectiveGasPrice': 20000000000,
 'from': '0x1a5022ed8d0a1597217928064d97b3f341cec0c4',
 'gasUsed': 5642425,
 'logsBloom': '0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 'status': 1,
 'to': None,
 'transactionHash': '0xbe75615740e3b8dbccb4cab70f380cb04ca45bfa252c1605c01683b74438b2b4',
 'transactionIndex': 1,
 'type': 

In [103]:
contract_instance = w3.eth.contract(address="0xEE4d4cFd1398dd2c9c5617cDB54cc6057FA30bd6", abi=abi)
nonce = w3.eth.getTransactionCount(public_key)

txn_dict = {
        'from': public_key,
        'chainId': CHAIN_ID,
        'gasPrice': w3.toWei('10', 'gwei'),
        'nonce': nonce,
        }
contract_txn =  contract_instance.functions.setConsentCode("0x1B9961cD860e2086AA24783F5f779785831B2e03").buildTransaction(txn_dict)
tx_receipt = sign_and_send(contract_txn, private_key, "change number")

MismatchedABI: ("The function 'setConsentCode' was not found in this contract's abi. ", 'Are you sure you provided the correct contract abi?')

In [259]:
tx_receipt

{'blockHash': '0x762ec1808d584b18ba33d60effe03e89e7b541ebc8ec6340da77e022f177de31',
 'blockNumber': 10070698,
 'contractAddress': None,
 'cumulativeGasUsed': 212450,
 'effectiveGasPrice': 10000000000,
 'from': '0x1a5022ed8d0a1597217928064d97b3f341cec0c4',
 'gasUsed': 44941,
 'logs': [AttributeDict({'address': '0x6fFa93601600582D9777D24584DB9405aAc0Be77',
   'blockHash': HexBytes('0x762ec1808d584b18ba33d60effe03e89e7b541ebc8ec6340da77e022f177de31'),
   'blockNumber': 10070698,
   'data': '0x0000000000000000000000000000000000000000000000000000000000000002',
   'logIndex': 3,
   'removed': False,
   'topics': [HexBytes('0x46dee1c9f4eec03602b071a8fa0b6fc2acebdc64f3606375a8545bf08510ee99')],
   'transactionHash': HexBytes('0x61ba8c71cd7a0373da551067ba7c735c08b0d182807a43a63b1fe0a2cab351f4'),
   'transactionIndex': 2})],
 'logsBloom': '0x000002000000000000000000000000000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000008000000000000000000000000

In [234]:
contract_instance = w3.eth.contract(address="0xd79Ee34C7d0DF71AFE54F59455BB25F3675A57ba", abi=abi)

contract_instance.functions.getNumber().call()

2

TypeError: 'Version' object is not callable

In [59]:
# Deploy contract and obtain transaction hash
tx_hash = my_contract.deploy()

/home/vagrant/miniconda/envs/luce_vm/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact
  


ValueError: {'code': -32601, 'message': 'The method eth_sendTransaction does not exist/is not available'}

This does work however this approach is deprecated..  
Even though the web3.py documentation itself still uses this approach in their examples section.

Method 2:

In [37]:
def deploy_contract(w3, contract_interface):
    tx_hash = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']).deploy()

    address = w3.eth.getTransactionReceipt(tx_hash)['contractAddress']
    return address

In [38]:
address = deploy_contract(w3, contract_interface)

/home/vagrant/miniconda/envs/luce_vm/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: deploy is deprecated in favor of contract.constructor.transact
  after removing the cwd from sys.path.


ValidationError: The field extraData is 97 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer http://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority for more details. The full extraData is: HexBytes('0xd883010a0e846765746888676f312e31372e33856c696e757800000000000000a34c84aa913eed9380f52f4687806b1d74932ceeeec3508e1234b936f12963d41cc42088973973e11c278a1652bdbeecaf51377f33d8d9b0c8c535c03b6f88c701')

In [39]:
print("Deployed {0} to: {1}\n".format(contract_id, address))

Deployed <stdin>:StoreVar to: 0x44bf10B9B66Df43Af722dF5f308fa16445efd34e



This also works but again with deprecation warning.

### Interact with Contract

In [40]:
# Create interface to deployed contract
deployed_contract = w3.eth.contract(
   address=address,
   abi=contract_interface['abi'])

**Estimate gas usage**

In [41]:
# Estimate gas usage for function call
gas_estimate = deployed_contract.functions.setVar(255).estimateGas()
print("Gas estimate to transact with setVar: {0}\n".format(gas_estimate))

Gas estimate to transact with setVar: 43133



In [42]:
def wait_for_receipt(w3, tx_hash, poll_interval):
    while True:
        tx_receipt = w3.eth.getTransactionReceipt(tx_hash)
        if tx_receipt:
            return tx_receipt
        time.sleep(poll_interval)

In [43]:
# Execute function call to smart contract
if gas_estimate < 100000:
    print("Sending transaction to setVar(255)\n")
    tx_hash = deployed_contract.functions.setVar(255).transact()
    receipt = wait_for_receipt(w3, tx_hash, 1)
    print("Transaction receipt mined: \n")
    pprint.pprint(dict(receipt))
else:
    print("Gas cost exceeds 100000")

Sending transaction to setVar(255)

Transaction receipt mined: 

{'blockHash': HexBytes('0xbf673fcce4ff3edd73e8cb4f3fe93abffa68b0c7fbbcfef1546ca792030e3ded'),
 'blockNumber': 9,
 'contractAddress': None,
 'cumulativeGasUsed': 43133,
 'from': '0x92d44e8579620f2db88a12e70fe38e8cdb3541ba',
 'gasUsed': 43133,
 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0xccaad3f56a637ac0257976e9593ee847384bac6008ca9b605b2c0a414de3af4d'), 'blockHash': HexBytes('0xbf673fcce4ff3edd73e8cb4f3fe93abffa68b0c7fbbcfef1546ca792030e3ded'), 'blockNumber': 9, 'address': '0x44bf10B9B66Df43Af722dF5f308fa16445efd34e', 'data': '0x', 'topics': [HexBytes('0x6c2b4666ba8da5a95717621d879a77de725f3d816709b9cbe9f059b8f875e284'), HexBytes('0x00000000000000000000000000000000000000000000000000000000000000ff')], 'type': 'mined'})],
 'logsBloom': HexBytes('0x00000400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000

In [44]:
# Call setVar on contract
deployed_contract.functions.setVar(4).transact()

HexBytes('0xa3943b7a6c0dae486bdb8449cd8fc4addc34855035f8ce69f276f82ba7ef5008')

In [45]:
# Obtain value from getVar on contract
deployed_contract.functions.getVar().call()

4

### New Contract Deployment Approach
Since the old way to deploy contracts via `w3.eth.contract.deploy()` is deprecated I also try out another pathway, as outlined in the contract section of the web3.py documentation [here](https://web3py.readthedocs.io/en/stable/contracts.html?highlight=abi).

Note: I replace solc with solcx since solc is throwing errors during compilation. solcx can compile the contract.  
Update: I am still using solcx instead of solc but LuceVM now already has all required packages installed

In [46]:
#!{sys.executable} -m pip install -U web3[tester]

There is a requirement conflict with eth-abi when trying to install web3tester. See if we can use ganache as sandbox instead.

In [47]:
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

In [48]:
# Solidity source code
contract_source_code = '''
pragma solidity ^0.4.21;

contract Greeter {
    string public greeting;

    function Greeter() public {
        greeting = 'Hello';
    }

    function setGreeting(string _greeting) public {
        greeting = _greeting;
    }

    function greet() view public returns (string) {
        return greeting;
    }
}
'''

In [49]:
compiled_sol = compile_source(contract_source_code) # Compiled source code
contract_interface = compiled_sol['<stdin>:Greeter']

The compilation works fine when using solcx instead of solc.

In [50]:
# web3.py instance
# w3 = Web3(Web3.EthereumTesterProvider())
# This causes a requirement conflix with eth-abi.. Use Ganache instead

In [51]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [52]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

**Instantiate Contract Object**

In [53]:
# Instantiate contract
Greeter = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])

**Deploy Contract**

In [54]:
# Submit the transaction that deploys the contract
tx_hash = Greeter.constructor().transact()

Halleluhjah! This worked :D  
The contract is deployed.  
And this time without a deprecation warning.  
So this is the new recommended approach of deployment.

In [55]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [56]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x89f4e30a53cc127a8d97970e935670d7fa2baccb4cc7a7dfca44b68a122e9bbd'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x2450062af4a29239dc844cb4734c75c3f98f248a676895e5768785c2c085f1c3'),
 'blockNumber': 11,
 'from': '0x92d44e8579620f2db88a12e70fe38e8cdb3541ba',
 'to': None,
 'gasUsed': 364124,
 'cumulativeGasUsed': 364124,
 'contractAddress': '0xa832b74da9969b9019d598d84E82Dd44F855D093',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [57]:
# Create the contract instance with the newly-deployed address
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

**Read contract state**

In [58]:
# Display the default greeting from the contract
print('Default contract greeting: {}'.format(
    greeter.functions.greet().call()
))

Default contract greeting: Hello


In [59]:
greeter.functions.abi

[{'constant': False,
  'inputs': [{'name': '_greeting', 'type': 'string'}],
  'name': 'setGreeting',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greet',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': True,
  'inputs': [],
  'name': 'greeting',
  'outputs': [{'name': '', 'type': 'string'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'constructor'}]

In [60]:
greeter.functions.greet().call()

'Hello'

**Change contract state**

In [61]:
tx_hash = greeter.functions.setGreeting('new greeting').transact()

In [62]:
# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0xa1e4b776a60b2ee8c25c5e981c729554fce166301bf63b78ca819db2d24122f3'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x6419793146fc9aea9697b55131ceeb73116abb4771881f64bc314847e887eae5'),
 'blockNumber': 12,
 'from': '0x92d44e8579620f2db88a12e70fe38e8cdb3541ba',
 'to': '0xa832b74da9969b9019d598d84e82dd44f855d093',
 'gasUsed': 33516,
 'cumulativeGasUsed': 33516,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [63]:
greeter.functions.greet().call()

'new greeting'

All is working as intended! I am very happy. This gives enough foundation. The examples here can be used as reference for how to interact with LUCE contract functions.

### Other Useful Snippets from web3 documentation

**web3.eth.*account* object**

In [64]:
# Create a new account
test_account = w3.eth.account.create()

In [65]:
# Display address
test_account.address

'0x27db5EFD8b6863F6E573fbFc04b7957a9F9d14C3'

In [66]:
# Show private key
test_account.privateKey

HexBytes('0xd6d5a9f687f225d069c80a3da11aadfd4b3b937dba40c9a4343b7eab0b69d35d')

In [69]:
# Restore account from private key
test_account_restored = w3.eth.account.privateKeyToAccount("0xd6d5a9f687f225d069c80a3da11aadfd4b3b937dba40c9a4343b7eab0b69d35d")

In [70]:
# Check if indeed address of restored account is the same
test_account_restored.address == test_account.address

True

In [71]:
type(test_account)

eth_account.signers.local.LocalAccount

**Sign a transaction**

In [72]:
# 1) First define a transaction dictionary
tx_dic ={
    'to': '0xF0109fC8DF283027b6285cc889F5aA624EaC1F55',
    'value': 1000000000,
    'gas': 2000000,
    'gasPrice': w3.toWei('40', 'gwei'),
    'nonce': w3.eth.getTransactionCount(test_account.address)
    
}

In [73]:
# 2) Then sign
test_account.signTransaction(tx_dic)

AttributeDict({'rawTransaction': HexBytes('0xf869808509502f9000831e848094f0109fc8df283027b6285cc889f5aa624eac1f55843b9aca00801ca0604dc371ee5871b5f7c932f598a769b324ef410690ed5aa594bb06828f8b0be6a0662d57f627a1b0939c1b2e37ca18342ec054acc3455f3afbcb56eed89eb3431e'),
 'hash': HexBytes('0x082e4663909a4e423663cd04d8e576fd56f135487a23bffe6aefbbdcdb49fb32'),
 'r': 43559429600093815668785665617397752938607520725881376783969806704001539574758,
 's': 46216025761660944488180574795730454324155410965290607253559946055926425010974,
 'v': 28})

---

**web3.eth.*contract* object**

In [74]:
test_contract = w3.eth.contract()

In [75]:
type(test_contract)

web3.utils.datatypes.PropertyCheckingFactory

In [76]:
test_contract.address

In [77]:
test_contract.abi

In [78]:
test_contract.bytecode

In [79]:
test_contract.deploy

<bound method Contract.deploy of <class 'web3.utils.datatypes.Contract'>>

In [80]:
test_function = test_contract.functions

In [81]:
test_function

### Drop Things Here

In [ ]:
# Contract address from Ganache
contract_address     = "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"